# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 57 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.09778
extracting tarball to tmp_2601.09778...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


Found 108 bibliographic references in tmp_2601.09778/main.bbl.
Error retrieving bib data for gordon_QuantitativeComparisonSmall_2003: 'author'
Error retrieving bib data for chira_RevisitingXraytoUVRelation_2026: 'author'
Error retrieving bib data for jin_CombinedOpticalXray_2012: 'author'
Error retrieving bib data for degraaff_LittleRedDots_2025: 'author'
Error retrieving bib data for degraaff_LittleRedDots_2025: 'author'
Error retrieving bib data for vandenberk_CompositeQuasarSpectra_2001: 'author'
Error retrieving bib data for glikman_NearInfraredSpectralTemplate_2006: 'author'
Error retrieving bib data for yue_StackingXRayObservations_2024: 'author'
Error retrieving bib data for fu_DiscoveryTwoLittle_2025: 'author'
Retrieving document from  https://arxiv.org/e-print/2601.10631
extracting tarball to tmp_2601.10631...

...

Retrieving document from  https://arxiv.org/e-print/2601.10666


extracting tarball to tmp_2601.10666...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.10709


extracting tarball to tmp_2601.10709...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.09778-b31b1b.svg)](https://arxiv.org/abs/2601.09778) | **The X-Ray Dot: Exotic Dust or a Late-Stage Little Red Dot?**  |
|| R. E. Hviding, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2026-01-16*|
|*Comments*| *21 pages, 7+2 Figures, 1+3 Tables, submitted to AJL. Comments welcome!*|
|**Abstract**|            JWST's "Little Red Dots" (LRDs) are increasingly interpreted as active galactic nuclei (AGN) obscured by dense thermalized gas rather than dust as evidenced by their X-ray weakness, blackbody-like continua, and Balmer line profiles. A key question is how LRDs connect to standard UV-luminous AGN and whether transitional phases exist and if they are observable. We present the "X-Ray Dot" (XRD), a compact source at $z=3.28$ observed by the NIRSpec WIDE GTO survey. The XRD exhibits LRD hallmarks: a blackbody-like ($T_{\rm eff} \simeq 6400\,$K) red continuum, a faint but blue rest-UV excess, falling mid-IR emission, and broad Balmer lines ($\rm FWHM \sim 2700-3200\,km\,s^{-1}$). Unlike LRDs, however, it is remarkably X-ray luminous ($L_\textrm{2$-$10$\,$keV} = 10^{44.18}\,$erg$\,$s$^{-1}$) and has a continuum inflection that is bluewards of the Balmer limit. We find that the red rest-optical and blue mid-IR continuum cannot be reproduced by standard dust-attenuated AGN models without invoking extremely steep extinction curves, nor can the weak mid-IR emission be reconciled with well-established X-ray--torus scaling relations. We therefore consider an alternative scenario: the XRD may be an LRD in transition, where the gas envelope dominates the optical continuum but optically thin sightlines allow X-rays to escape. The XRD may thus provide a physical link between LRDs and standard AGN, offering direct evidence that LRDs are powered by supermassive black holes and providing insight into their accretion properties.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.10631-b31b1b.svg)](https://arxiv.org/abs/2601.10631) | **Circumplanetary Disk Candidate in the Disk of HD 163296 Traced by Localized Emission from Simple Organics**  |
|| A. F. Izquierdo, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2026-01-16*|
|*Comments*| *18 pages, 11 figures, 2 tables. Accepted for publication in ApJL*|
|**Abstract**|            Atacama Large Millimeter/submillimeter Array observations suggest that the disc of HD 163296 is being actively shaped by embedded, yet unseen protoplanets, as indicated by numerous gas and dust substructures consistent with planet-disc interaction models. We report the first detection of simple organic molecules, HCN and C2H, tracing a candidate circumplanetary disc (CPD) in the HD 163296 system, located at an orbital radius of $R=88\pm7$ au and azimuth $\phi=46\pm3^\circ$ (or $R=0.75''$, $\rm{PA}=350^\circ$ in projected sky coordinates), and originating near the midplane of the circumstellar disc. The signature is localised but spectrally resolved, and it overlaps with a previously reported planet candidate, P94, identified through kinematic perturbations traced by CO lines. We propose a scenario in which the observed chemical anomalies arise from increased heating driven by the forming planet and ongoing accretion through its CPD, facilitating the thermal desorption of species that would otherwise remain frozen out in the disc midplane, and potentially triggering the activation barriers of chemical reactions that lead to enhanced molecular production. Based on a first-order dynamical analysis of the HCN spectrum from the CPD--isolated with a 7$\sigma$ significance--we infer an upper limit on the planet mass of 1.8 $M_{\rm Jup}$, consistent with predictions from CO kinematics and constraints from direct imaging studies. By comparing the CPD sizes derived from our models with theoretical expectations where the CPD radius corresponds to roughly one-third of the planet's Hill radius, we favor CPD gas temperatures $T > 150$ K, planet masses $M_{\rm p} < 1.0$ $M_{\rm Jup}$, and CPD radii $R_{\rm CPD} < 2$ au.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.10666-b31b1b.svg)](https://arxiv.org/abs/2601.10666) | **Observation Timelines for the Potential Lunar Impact of Asteroid 2024 YR4**  |
|| Y. He, et al. -- incl., <mark>Y. Wu</mark> |
|*Appeared on*| *2026-01-16*|
|*Comments*| **|
|**Abstract**|            The near-Earth asteroid 2024 YR4 -- a $\sim$60 m rocky object that was once considered a potential Earth impactor -- has since been ruled out for Earth but retained a $\sim$4.3% probability of striking the Moon in 2032. Such an impact, with equivalent kinetic energy of $\sim$6.5 Mt TNT, is expected to produce a $\sim$1 km crater on the Moon, and will be the most energetic lunar impact event ever recorded in human history. Despite the associated risk, this scenario offers a rare and valuable scientific opportunity. Using a hybrid framework combining Monte Carlo orbital propagation, smoothed particle hydrodynamics (SPH) impact modeling, and N-body ejecta dynamics, we evaluate the physical outcomes and propose the observation timelines of this rare event. Our results suggest an optical flash of visual magnitude from -2.5 to -3 lasting several minutes directly after the impact, followed by hours of infrared afterglow from $\sim$2000 K molten rock cooling to a few hundred K. The associated seismic energy release would lead to a global-scale lunar reverberation (magnitude $\sim$5.0) that can be detectable by modern seismometers. Furthermore, the impact would eject $\sim$10$^8$ kg of debris that escapes the lunar gravity, with a small fraction reaching Earth to produce a lunar meteor outburst within 100 years. Finally, we integrate these results into a coordinated observation timeline, identifying the best detection windows for ground-based telescopes, lunar orbiters, and surface stations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.10709-b31b1b.svg)](https://arxiv.org/abs/2601.10709) | **Euclid preparation. 3D reconstruction of the cosmic web with simulated Euclid Deep spectroscopic samples**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-01-16*|
|*Comments*| *30 pages, 23 figures. Accepted for publication in A&A*|
|**Abstract**|            The ongoing Euclid mission aims to measure spectroscopic redshifts for approximately two million galaxies using the H $\alpha$ line emission detected in near-infrared slitless spectroscopic data from the Euclid Deep Fields (EDFs). These measurements will reach a flux limit of $5\times 10^{-17}\,{\rm erg}\,{\rm cm}^{-2}\,{\rm s}^{-1}$ in the redshift range $0.4<z<1.8$, opening the door to numerous investigations involving galaxy evolution, extending well beyond the mission's core objectives. The achieved H $\alpha$ luminosity depth will lead to a sufficiently high sampling, enabling the reconstruction of the large-scale galaxy environment. We assess the quality of the reconstruction of the galaxy cosmic web environment with the expected spectroscopic dataset in EDFs. The analysis is carried out on the Flagship and GAEA galaxy mock catalogues. The quality of the reconstruction is first evaluated using geometrical and topological statistics measured on the cosmic web, namely the length of filaments, the area of walls, the volume of voids, and its connectivity and multiplicity. We then quantify how accurately gradients in galaxy properties with distance from filaments can be recovered. As expected, the small-scale redshift-space distortions, have a strong impact on filament lengths and connectivity, but can be mitigated by compressing galaxy groups before skeleton extraction. The cosmic web reconstruction is biased when relying solely on H $\alpha$ emitters. This limitation can be mitigated by applying stellar mass weighting during the reconstruction. However, this approach introduces non-trivial biases that need to be accounted for when comparing to theoretical predictions. Redshift uncertainties pose the greatest challenge in recovering the expected dependence of galaxy properties, though the well-established stellar mass transverse gradients towards filaments can still be observed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.09778/figures/scatter.png', 'tmp_2601.09778/figures/postage.png', 'tmp_2601.09778/figures/example.png', 'tmp_2601.09778/figures/compare.png']
copying  tmp_2601.09778/figures/scatter.png to _build/html/
copying  tmp_2601.09778/figures/postage.png to _build/html/
copying  tmp_2601.09778/figures/example.png to _build/html/
copying  tmp_2601.09778/figures/compare.png to _build/html/
exported in  _build/html/2601.09778.md
    + _build/html/tmp_2601.09778/figures/scatter.png
    + _build/html/tmp_2601.09778/figures/postage.png
    + _build/html/tmp_2601.09778/figures/example.png
    + _build/html/tmp_2601.09778/figures/compare.png
found figures ['tmp_2601.10631/./fig_hd163296_cpd_spectrum2_v3.png', 'tmp_2601.10631/./hd163296_deprojected_hcn_isovelocities.png', 'tmp_2601.10631/./alldatasets_hf_hcn_c2h_jvm1_.png']
copying  tmp_2601.10631/./fig_hd163296_cpd_spectrum2_v3.png to _build/html/
copying  tmp_2601.10631/./hd163296_deprojected_hcn_isovelocities.png to _build/ht

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\todo}[1]{{\color{red}{#1}}}$</div>



<div id="title">

# The X-Ray Dot: Exotic Dust or a Late-Stage Little Red Dot?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.09778-b31b1b.svg)](https://arxiv.org/abs/2601.09778)<mark>Appeared on: 2026-01-16</mark> -  _21 pages, 7+2 Figures, 1+3 Tables, submitted to AJL. Comments welcome!_

</div>
<div id="authors">

R. E. Hviding, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** JWST's "Little Red Dots" (LRDs) are increasingly interpreted as active galactic nuclei (AGN) obscured by dense thermalized gas rather than dust as evidenced by their X-ray weakness, blackbody-like continua, and Balmer line profiles.A key question is how LRDs connect to standard UV-luminous AGN and whether transitional phases exist and if they are observable.We present the "X-Ray Dot" (XRD), a compact source at $z=3.28$ observed by the NIRSpec WIDE GTO survey.The XRD exhibits LRD hallmarks: a blackbody-like ( $T_{\rm eff} \simeq 6400$ K) red continuum, a faint but blue rest-UV excess, falling mid-IR emission, and broad Balmer lines ( $\rm FWHM \sim 2700-3200 km s^{-1}$ ).Unlike LRDs, however, it is remarkably X-ray luminous ( $L_\textrm{2$-$10 keV} = 10^{44.18}$ erg s $^{-1}$ ) and has a continuum inflection that is bluewards of the Balmer limit.We find that the red rest-optical and blue mid-IR continuum cannot be reproduced by standard dust-attenuated AGN models without invoking extremely steep extinction curves, nor can the weak mid-IR emission be reconciled with well-established X-ray--torus scaling relations.We therefore consider an alternative scenario: the XRD may be an LRD in transition, where the gas envelope dominates the optical continuum but optically thin sightlines allow X-rays to escape.The XRD may thus provide a physical link between LRDs and standard AGN, offering direct evidence that LRDs are powered by supermassive black holes and providing insight into their accretion properties.

</div>

<div id="div_fig1">

<img src="tmp_2601.09778/figures/scatter.png" alt="Fig6" width="100%"/>

**Figure 6. -** 
    Comparison of the XRD to the population of LRDs (dark gray) and non-LRD broad-line objects (light gray) from degraaff_LittleRedDots_2025 based on emission line and modified black-body fitting.
    Panels show: Blackbody temperature vs. luminosity (top left); Balmer decrement vs. H$\alpha$ luminosity (top middle); H$\alpha$ luminosity vs. UV magnitude (top right); H$\alpha$ luminosity vs. Blackbody luminosity (bottom left); O {\sc i}$\lambda$8446Å luminosity vs. H$\alpha$ luminosity (bottom middle); H$\alpha$ EW vs. UV power-law slope (bottom right).
    The XRD generally lies in regions occupied by LRDs and not non-LRD broad line galaxies and on the relations derived in degraaff_LittleRedDots_2025.
     (*fig:scatter*)

</div>
<div id="div_fig2">

<img src="tmp_2601.09778/figures/postage.png" alt="Fig2.1" width="50%"/><img src="tmp_2601.09778/figures/example.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** 
    Overview of observations for the X-Ray Dot (XRD).
    Top panels present the space-based imaging compiled from the 3DHST survey from HST and Spitzer, the RGB composite from HST, and the hard and soft X-ray cutouts from Chandra.
    HST/Spitzer panels show the $4"\times4"$ cutout in the band, magnitude, and S/N whereas the Chandra panels show $8"\times8"$ cutouts.
    The bottom panel compares the XRD photometry and scaled spectrum (black) to an LRD with a similar rest-optical continuum (RUBIES$-$UDS$-$144195, red) and to the STScI composite quasar spectrum derived from vandenberk_CompositeQuasarSpectra_2001 and glikman_NearInfraredSpectralTemplate_2006 shown unattenuated (blue) and extincted (purple; $\rm A_{V}=1.37$) by an SMC-like dust law  (gordon_QuantitativeComparisonSmall_2003) .
    Comparison spectra are normalized to the XRD scaled spectrum at rest 5500Å.
     (*fig:xrd*)

</div>
<div id="div_fig3">

<img src="tmp_2601.09778/figures/compare.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
        Top: The rest-frame SED of the XRD (black) compared to RUBIES-BLAGN-1 \citep[green;][]{wang_RUBIESJWSTNIRSpec_2025} and Forge I and II \citep[red and blue respectively;][]{fu_DiscoveryTwoLittle_2025}.
        Bottom Left:
        $L_{\rm 2,keV}$ versus $L_{2500\rm\mathring{A}}$ for the XRD, the Forges, and the X-ray upper limits for LRDs nondetections from yue_StackingXRayObservations_2024(grey triangles).
        The XRD and Forge sources lie well above the standard $z\simeq0-3$ AGN relation  (chira_RevisitingXraytoUVRelation_2026) , indicating they are X-ray luminous for their UV emission.
        Applying a dust correction (\S\ref{ssec:continuum}) to the XRD, along with the corrections for the Forges in fu_DiscoveryTwoLittle_2025, brings all three sources closer to the relation.
        Bottom Right: Comparison of X-ray and H$\alpha$ luminosity.
        The measured XRD properties follow the relation for $z<0.4$ unobscured AGN  (jin_CombinedOpticalXray_2012) , but correcting the H$\alpha$ luminosity for dust in the same manner as $L_{2500\rm\mathring{A}}$ produces an H$\alpha$ luminosity that is overluminous for its X-ray emission.
     (*fig:compare*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.09778"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Mj}{M_{\rm Jup}}$
$\newcommand{\phip}{\phi_{p}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\twCOfull}{^{12}CO J=2\rightarrow1}$
$\newcommand{\twCO}{^{12}CO}$
$\newcommand{\thCO}{^{13}CO}$
$\newcommand{\thCOfull}{^{13}CO J=2\rightarrow1}$
$\newcommand{\eiCO}{C^{18}O}$
$\newcommand{\eiCOfull}{C^{18}O J=2\rightarrow1}$
$\newcommand{\CSfull}{CS J=7\rightarrow6}$
$\newcommand{\CtwoH}{C_{\rm 2}H}$
$\newcommand{\CtwoHfull}{C_{\rm 2}H N=3\rightarrow2 J=5/2\rightarrow3/2}$
$\newcommand{\hcnfull}{HCN J=3\rightarrow2}$
$\newcommand{\Br}{Br\gamma}$
$\newcommand{\emcee}{\textsc{emcee}}$
$\newcommand{\discminer}{\textsc{discminer}}$
$\newcommand{\disksurf}{\textsc{disksurf}}$
$\newcommand{\exoalma}{exo\textsc{alma}}$
$\newcommand{\filfinder}{\textsc{FilFinder}}$
$\newcommand{\mwc}{MWC 480}$
$\newcommand{\hd}{HD 163296}$
$\newcommand{\as}{AS 209}$
$\newcommand{\im}{IM Lup}$
$\newcommand{\gm}{GM Aur}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

#   $\begin{minipage}{0.95\textwidth}    \centering    \large\bfseries    Circumplanetary Disk Candidate in the Disk of HD 163296 Traced by Localized Emission from Simple Organics  \end{minipage}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.10631-b31b1b.svg)](https://arxiv.org/abs/2601.10631)<mark>Appeared on: 2026-01-16</mark> -  _18 pages, 11 figures, 2 tables. Accepted for publication in ApJL_

</div>
<div id="authors">

A. F. Izquierdo, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Atacama Large Millimeter/submillimeter Array observations suggest that the disc of $\hd$ is being actively shaped by embedded, yet unseen protoplanets, as indicated by numerous gas and dust substructures consistent with planet-disc interaction models. We report the first detection of simple organic molecules, HCN and $\CtwoH$ , tracing a candidate circumplanetary disc (CPD) in the $\hd$ system, located at an orbital radius of $R=88\pm7$ au and azimuth $\phi=46\pm3^\circ$ (or $R=0\farcs{75}$ , $\rm{{PA}}=350^\circ$ in projected sky coordinates), and originating near the midplane of the circumstellar disc. The signature is localised but spectrally resolved, and it overlaps with a previously reported planet candidate, P94, identified through kinematic perturbations traced by CO lines. We propose a scenario in which the observed chemical anomalies arise from increased heating driven by the forming planet and ongoing accretion through its CPD, facilitating the thermal desorption of species that would otherwise remain frozen out in the disc midplane, and potentially triggering the activation barriers of chemical reactions that lead to enhanced molecular production. Based on a first-order dynamical analysis of the HCN spectrum from the CPD---isolated with a 7 $\sigma$ significance---we infer an upper limit on the planet mass of 1.8 $\Mj$ , consistent with predictions from CO kinematics and constraints from direct imaging studies. By comparing the CPD sizes derived from our models with theoretical expectations where the CPD radius corresponds to roughly one-third of the planet’s Hill radius, we favor CPD gas temperatures $T > 150$ K, planet masses $M_{\rm p} < 1.0$ $\Mj$ , and CPD radii $R_{\rm CPD} < 2$ au.

</div>

<div id="div_fig1">

<img src="tmp_2601.10631/./fig_hd163296_cpd_spectrum2_v3.png" alt="Fig6" width="100%"/>

**Figure 6. -** Illustration of the folding procedure used to isolate the $\hcnfull$  spectra from the localised signal around the kinematic planet P94, possibly tracing circumplanetary disc (CPD) emission. The blueshifted side of the circumstellar disc (CSD) is spectrally mirrored and subtracted from the redshifted side, where the planet resides, retaining the excess intensity contribution from the candidate CPD. The bottom panel shows the resulting folded spectra extracted from a $7\times7$ pixel box ($\sim\!2$ beams per side) centred on the proposed CPD location at $R = 0$\farcs${75}$ and $\mathrm{PA} = 350^\circ$. Gaussian fits to the brightest 20\% of the spectra are overlaid as orange lines. The global peak profile has an amplitude of 5.9 mJy beam$^{-1}$ and a centroid velocity of 2.29 $\kms$  marked by the vertical dashed line, measured with respect to the inferred systemic velocity of the CSD.
               (*fig:channels_mirror*)

</div>
<div id="div_fig2">

<img src="tmp_2601.10631/./hd163296_deprojected_hcn_isovelocities.png" alt="Fig8" width="100%"/>

**Figure 8. -** Deprojected view of the disc of $\hd$  as traced by HCN $J=3\rightarrow2$ emission in the 2.3 $\kms$  velocity channel relative to the circumstellar disc’s $\upsilon_{\rm LSRK}$, assuming elevated (left panel) and midplane (right panel) emission heights. Overlaid are Keplerian isovelocity curves at values around the inferred CPD centroid velocity ($\sim\!2.3$\kms ; see Sect. \ref{subsec:cpd} for details). The observed CPD emission is consistent with a midplane origin, as indicated by its alignment with the isovelocity curve (yellow line) at the CPD centroid velocity when a $z/r = 0$ is adopted for the channel‑map deprojection. The right panel also includes $C_2$H emission (in purple contours) at the same velocity channel, with the CPD signal overlapping that seen in HCN.
               (*fig:isovelocities_deproj*)

</div>
<div id="div_fig3">

<img src="tmp_2601.10631/./alldatasets_hf_hcn_c2h_jvm1_.png" alt="Fig9" width="100%"/>

**Figure 9. -** Selected intensity maps for $\hcnfull$  and $\CtwoH$full$F=3\rightarrow2$ lines, shown at mirrored (blue) and around (red) velocity channels where a localised signal overlaps with the kinematic planet candidate P94 first proposed by  ([Izquierdo, et. al 2022](https://ui.adsabs.harvard.edu/abs/2022ApJ...928....2I), [Izquierdo, et. al 2023](https://ui.adsabs.harvard.edu/abs/2023A&A...674A.113I)) , providing unprecedented evidence of circumplanetary material orbiting a still-forming planet (see Sect. \ref{subsec:cpd}). A guiding dashed box is centred at the inferred location of the signal, at a projected separation of $R = 0$\farcs${75}$ from the image centre and a position angle of $\mathrm{PA} = 350^\circ$, measured from north through east. Although the hf1, hf2, and hf3 cubes for the HCN intensity channels correspond to datasets imaged at the central frequencies of different hyperfine components in the $J=3\rightarrow2$ group, the bulk of the emission in all cases originates from the dominant $F=4\rightarrow3$ line, blended with the $F=2\rightarrow1$ and $F=3\rightarrow2$ components. Contours enclose emission above five times the rms noise level of 0.43 mJy beam$^{-1}$ for HCN and 0.53 mJy beam$^{-1}$ for $C_2$H.
               (*fig:allchannels*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.10631"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

113  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
